In [124]:
import pandas as pd
import csv
import cv2
import os
import matplotlib.pyplot as plt

import numpy as np
import cv2

# Question 2 :
## Generate student.csv

In [125]:
directory_path = []

student_id = []
sur_name = []
first_name = []
code = []

for filename in os.listdir('Data Folder'):
    f = os.path.join('Data Folder',filename)
    if os.path.isfile(f) and f.endswith(".jpg"):
        directory_path.append(f)
        student_id.append(f[12:19])
        sur_name.append(f[20:-8])
        first_name.append(f[-8])
        code.append(f[-6:-4])

In [126]:
student = open('Data Folder/student', 'w', encoding='UTF8', newline='')
writer = csv.writer(student)

writer.writerow(['StudentId', 'Surname', 'Firstname', 'Code'])
for i in range(len(student_id)):
    line = [student_id[i], sur_name[i], first_name[i], code[i]]
    writer.writerow(line)
student.close()

In [127]:
data = pd.read_csv('Data Folder/student', header=0, index_col=False)

In [128]:
df = pd.DataFrame(data=data)

In [129]:
df

,StudentId,Surname,Firstname,Code
0,2000102,NguyenVan,A,3A
1,2000103,NguyenVan,B,3A
2,2000104,NguyenVan,C,3A


# Question 3 :
## Generate the first 5 answers of one student

In [130]:
red = (0, 0, 255)
black = (0, 0, 0)
white = (255, 255, 255)

In [131]:
def show_images(titles, images, wait=True):
    for (title, image) in zip(titles, images):
        cv2.imshow(title, image)
    if wait:
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [132]:
def take_ques(x):
    if x < 30:
        return 0
    elif x < 65:
        return 1
    elif x < 100:
        return 2
    elif x < 130:
        return 3
    else:
        return 4

In [133]:
# Generate answer for test

img = plt.imread('Data Folder/2000102_NguyenVanA_3A.jpg')

img_crop = img[120:225,90:255,:]

gray_img = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)

contours, _ = cv2.findContours(gray_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(img_crop, contours, -1, red, 3)

answer_5 = [-1]*5

for cnt in contours:
    for i in cnt:
        x = i[0][0]
        y = i[0][1]
        if y < 23:
            index = 0
        elif y < 40:
            index = 1
        elif y < 58:
            index = 2
        elif y < 78:
            index = 3
        else:
            index = 4
        if answer_5[index] != -1: continue
        answer_5[index] = take_ques(x)

In [134]:
# Generate result for test

img = plt.imread('Answer Folder/3A.jpg')

img_crop = img[120:225,90:255,:]

gray_img = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)

contours, _ = cv2.findContours(gray_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(img_crop, contours, -1, red, 3)

result_5 = [-1]*5

for cnt in contours:
    for i in cnt:
        x = i[0][0]
        y = i[0][1]
        if y < 23:
            index = 0
        elif y < 40:
            index = 1
        elif y < 58:
            index = 2
        elif y < 78:
            index = 3
        else:
            index = 4
        if result_5[index] != -1: continue
        result_5[index] = take_ques(x)

In [135]:
grade_5 = 0
print('Answer : ', answer_5)
print('Result : ', result_5)

for i in range(5):
    if answer_5[i] == result_5[i]:
        grade_5 += 1
print('Student - NguyenVanA : ', grade_5)

Answer :  [0, 3, 1, 1, 4]
Result :  [0, 2, 1, 3, 4]
Student - NguyenVanA :  3


# Question 4 :
## Generate all answers of one student

In [136]:
difficult_ques = [0]*60

In [137]:
def get_image_crop(img, row, col):
    return img[(120+110*row):(225+110*row),(90+285*col):(255+285*col),:]

In [138]:
def get_abcde(x):
    if x < 30:
        return 0
    elif x < 65:
        return 1
    elif x < 100:
        return 2
    elif x < 130:
        return 3
    else:
        return 4

In [139]:
def get_5_answers(contours):
    answer_5 = [-1]*5
    for cnt in contours:
        for i in cnt:
            x = i[0][0]
            y = i[0][1]
            if y < 23:
                index = 0
            elif y < 40:
                index = 1
            elif y < 58:
                index = 2
            elif y < 78:
                index = 3
            else:
                index = 4
            if answer_5[index] != -1: continue
            answer_5[index] = get_abcde(x)
    return answer_5

In [140]:
def get_answer(img):
    answer = []
    for i in range(12):
        if i < 6:
            img_crop = get_image_crop(img, i, 0)
        else:
            img_crop = get_image_crop(img, i-6, 1)
            
        gray_img = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
        contours, _ = cv2.findContours(gray_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(img_crop, contours, -1, red, 3)
    
        answer_5 = get_5_answers(contours)
        for i in answer_5:
            answer.append(i)
    return answer

In [141]:
# Generate result for test
img_result = plt.imread('Answer Folder/3A.jpg')
result = get_answer(img_result)

In [142]:
# Generate answer for test

def grading_stu(path_stu_img):
    img_answer = plt.imread(str(path_stu_img))
    answer = get_answer(img_answer)

    grade = 0
    for i in range(60):
        if answer[i] == result[i]:
            grade += 1
        else:
            difficult_ques[i] += 1
    return grade

In [143]:
print('Student - NguyenVanA : ', grading_stu('Data Folder/2000102_NguyenVanA_3A.jpg'), '/ 60')

Student - NguyenVanA :  15 / 60


# Question 5 :
## Generate grading.csv

In [144]:
grading = open('Data Folder/grading', 'w', encoding='UTF8', newline='')
writer = csv.writer(grading)

writer.writerow(['StudentId', 'Grade'])
for i in range(len(student_id)):
    line = [student_id[i]]
    writer.writerow(line)
grading.close()

In [145]:
data_grade = pd.read_csv('Data Folder/grading', header=0, index_col=False)

In [146]:
df_grade = pd.DataFrame(data=data_grade)

In [147]:
df_grade

,StudentId,Grade
0,2000102,NaN
1,2000103,NaN
2,2000104,NaN


In [148]:
grade_all = []
pf_all = []

#grading(path_stu_img)
for i in directory_path:
    file = i.replace('\\', '/')
    grade = grading_stu(file)
    if grade > 30:
        pf_all.append('P')
    else:
        pf_all.append('F')
    grade_all.append(grade)

In [149]:
df_grade['Grade'] = grade_all

In [150]:
df_grade

,StudentId,Grade
0,2000102,15
1,2000103,60
2,2000104,12


# Question 6 :
## Summarize 3 most difficult questions

In [151]:
df_diff_ques = pd.DataFrame(columns=["Ques", "Num of false"])
df_diff_ques["Ques"] = [i for i in range(1,61)]
df_diff_ques["Num of false"] = difficult_ques

In [152]:
print('3 questions are most difficult: ')
df_diff_ques.nlargest(3, 'Num of false')

3 questions are most difficult: 


,Ques,Num of false
3,4,3
5,6,3
7,8,3


# Question 7 :
## Generate pass / fail

In [153]:
df_grade["P/F"] = pf_all

In [154]:
df_grade

,StudentId,Grade,P/F
0,2000102,15,F
1,2000103,60,P
2,2000104,12,F
